## Climbing Shoe Website Parser
## Website: TrackNTrail

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyshorteners
import pyperclip

Parse for max number of pages to iterate the parser through

In [18]:
TracknTrail_pages = 1

url = "https://shop.trackntrail.ca/footwear/climbing-shoes/page3.html"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
pager_row = soup.find("div", class_="pager row")
pager_row_elements = pager_row.find_all("li", class_="number")
for sale_element in pager_row_elements:
    if int(sale_element.find("a").text) > TracknTrail_pages:
        TracknTrail_pages = int(sale_element.find("a").text)

Parsing through each webpage and converting it into a dataframe

In [19]:
df = pd.DataFrame(columns=["Shoe","Sale Price","Original Price","Discount","URL","Shipping Cost"])

for i in range(1,TracknTrail_pages+1):
    url = "https://shop.trackntrail.ca/footwear/climbing-shoes/" + "page{}.html".format(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="products row")
    product_elements = results.find_all("div", class_="product col-xs-6 col-sm-3 col-md-3")

    for sale_element in product_elements:
        if sale_element.find("div", class_="right") != None:
            new_row = {"Shoe" : sale_element.find("div", class_="info").find("a").text.strip(),
            "Sale Price" : float(sale_element.find("div", class_="right").text.strip()[2:]),
            "Original Price" : float(sale_element.find("div", class_="left").text.strip()[2:]),
            "Discount" : str(round((float(sale_element.find("div", class_="left").text.strip()[2:])-float(sale_element.find("div", class_="right").text.strip()[2:]))/float(sale_element.find("div", class_="left").text.strip()[2:]),2)*100)+"%",
            "URL" : sale_element.find("a")["href"],
            "Shipping Cost" : 25
            }
            new_row_df = pd.DataFrame([new_row])
            df = pd.concat([df, new_row_df],ignore_index=True)


type_tiny = pyshorteners.Shortener() 
df["URL"] = df["URL"].apply(lambda x: type_tiny.tinyurl.short(x))
df["Post-Tax Price"] = (df["Sale Price"]+df["Shipping Cost"])*1.13

Output Table

In [20]:
#Output the table with values sorted
df.sort_values(by="Discount",ascending=False)

,Shoe,Sale Price,Original Price,Discount,URL,Shipping Cost,Post-Tax Price
0,The General,169.99,229.95,26.0%,https://tinyurl.com/2xtudhgv,25,220.3387
1,Zone LV Shoe,129.99,174.95,26.0%,https://tinyurl.com/27y8q3yq,25,175.1387
